In [19]:
import numpy as np
import tensorflow as tf
import math
import random
from os import listdir
import codecs
import nltk
import data
from gensim.models import Word2Vec

In [20]:
vocab_size = 324
d, id_to_token = data.get_data(vocab_size)
for k in id_to_token:
    if id_to_token[k] == 'eos':
        id_to_token[k] = '\n'

324


In [21]:
x, y = data.sample(d, 5)

In [22]:
hidden_size = 100
num_steps = 10
learning_rate = 0.01
num_runs_between_logging = 3000

In [23]:
Wxh = tf.Variable(tf.random_uniform([vocab_size, hidden_size], maxval=0.01)) 
Whh = tf.Variable(tf.random_uniform([hidden_size, hidden_size], maxval=0.01))
bh = tf.Variable(tf.zeros([hidden_size]))
Why = tf.Variable(tf.random_uniform([hidden_size, vocab_size], maxval=0.01)) 
by = tf.Variable(tf.zeros([vocab_size]))
U_z = tf.Variable(tf.random_uniform([vocab_size, hidden_size]))
W_z = tf.Variable(tf.random_uniform([hidden_size, hidden_size]))
U_r = tf.Variable(tf.random_uniform([vocab_size, hidden_size]))
W_r = tf.Variable(tf.random_uniform([hidden_size, hidden_size]))

In [24]:
inputs = [tf.placeholder(tf.float32, [1, vocab_size]) for _ in xrange(num_steps)]
keep_prob = tf.placeholder(tf.float32)
outputs = {}
hs = {}
hs[-1] = tf.placeholder(tf.float32, [1, hidden_size])
# GRU UNIT
for i in range(len(inputs)):
    r = tf.nn.softmax(tf.matmul(inputs[i], U_r) + tf.matmul(hs[i-1], W_r))
    z = tf.nn.softmax(tf.matmul(inputs[i], U_z) + tf.matmul(hs[i-1], W_z))
    h = tf.nn.tanh(tf.matmul(inputs[i], Wxh) + tf.matmul(hs[i - 1] * r, Whh))
    hs[i] = tf.mul(tf.sub(tf.ones_like(z), z), h) + tf.mul(z , hs[i - 1])
#     hs[i]      = tf.nn.tanh(tf.matmul(inputs[i], Wxh) + tf.matmul(hs[i-1], Whh) + bh)
    hs_drop = tf.nn.dropout(hs[i], keep_prob)
    outputs[i] = tf.nn.softmax(tf.matmul(hs_drop, Why) + by)
print inputs[0], inputs[1], hs[-1], hs[0]

Tensor("Placeholder_32:0", shape=(1, 324), dtype=float32) Tensor("Placeholder_33:0", shape=(1, 324), dtype=float32) Tensor("Placeholder_43:0", shape=(1, 100), dtype=float32) Tensor("add_53:0", shape=(1, 100), dtype=float32)


In [25]:
truths = [tf.placeholder(tf.float32, [None, vocab_size]) for i in xrange(num_steps)]
cross_entropies = [-tf.reduce_sum(truths[i]*tf.log(outputs[i])) for i in xrange(num_steps)]  # These operations act element-wise.
cross_entropy = tf.add_n(cross_entropies)

In [26]:
train_step = tf.train.AdagradOptimizer(learning_rate).minimize(cross_entropy)

In [27]:
init = tf.initialize_all_variables()
sess = tf.Session()
sess.run(init)

In [28]:
def hallucinate(sess, seed_ix, num_chars):
    ix = seed_ix
    ixes = [ix]
    hallucination_h_state = np.zeros([1, hidden_size]) 
    for i in range(num_chars):
        # Create a random starting letter 
        x_in, _ = data.sample(d, 1)
#         make_hot_vec([1,vocab_size], ix)

        feed_dict={inputs[0]: x_in}
        feed_dict.update({hs[-1]:hallucination_h_state})
        feed_dict.update({keep_prob: 1.0})

        output, hallucination_h_state = sess.run((outputs[0],hs[0]), feed_dict=feed_dict)
        probs = output[0]
        ix = np.random.choice(range(len(probs)), p=probs)
        ixes.append(ix)
    hallucination=' '.join([id_to_token[ix] for ix in ixes])
    print hallucination

In [29]:
losses = []
iterations = 0
ix = 0
h_state = np.zeros([1, hidden_size])
smooth_loss = -np.log(1.0/vocab_size)*num_steps # loss at iteration 0

In [30]:
def RunModel():
    global ix,losses,iterations,h_state,smooth_loss
    run = 0
    while True:
        letters=[]
        if run > 50000:
            break
        x, y = data.sample(d, num_steps)
        feed_dict={inputs[i]: x[i].reshape(1, -1) for i in range(num_steps)}
        feed_dict.update({truths[i]: y[i].reshape(1, -1) for i in range(num_steps)})
        feed_dict.update({hs[-1]:h_state})
        feed_dict.update({keep_prob: 0.8})

        _, h_state, loss_out = sess.run((train_step, hs[num_steps-1], cross_entropy), feed_dict=feed_dict)
        smooth_loss = smooth_loss * 0.999 + loss_out * 0.001
        ix += num_steps
        #        letters.append(data[ix])
        run += 1
        if run % num_runs_between_logging == 0:   
            iterations += 1
            #        print "train: ", ''.join(letters)
            print "========== Iteration ", run, " Loss: ", smooth_loss, " =============="
            losses.append(smooth_loss)
            print sess.run(outputs[0], feed_dict=feed_dict).shape
            hallucinate(sess, random.randint(0,vocab_size-1), 30)
            
RunModel()

========== Iteration  3000  Loss:  45.537040062  ==============
(1, 324)
voice Of enjoying break When thy ; her banish 
 dew through caked Open 
 to hawthorn-tree 
 Cloys 
 
 the 
 White With earth ploughboy weary and together Through
========== Iteration  6000  Loss:  36.5268465443  ==============
(1, 324)
hark ; How mossy 's 
 
 
 is 'll 
 buds , , bring.— same pleasures bubbles All ripe spread 
 and Too thought 
 with abroad Quickly One moment


KeyboardInterrupt: 

In [ ]:
hallucinate(sess, random.randint(0,vocab_size-1), 100)